In [1]:
import os
import argparse

from tqdm import tqdm

import numpy as np
import torch
from torch.utils.data import DataLoader

from onnxutils.common import OnnxModel
from onnxutils.onnx2torch import convert

In [2]:
dataset_path = '/mnt/edisk/dataset/unimodel_calibrate'
model_path = 'unimodel.optimized.onnx'
qmodel_path = 'unimodel.quantized.onnx'

# Prepare Model&Dataset

In [3]:
from onnxutils.common import DatasetUtils

from unimodel_pipeline import UnimodelDataset

In [4]:
onnx_model = OnnxModel.from_file(model_path)
onnx_model.topological_sort()
torch_model = convert(onnx_model)

dataset = UnimodelDataset(dataset_path, torch_model.onnx_mapping.inputs)

# Quantize Model

In [5]:
from torch.ao.quantization.observer import HistogramObserver, PerChannelMinMaxObserver
from torch.ao.quantization.fake_quantize import FakeQuantize

from onnxutils.quantization.utils import symbolic_trace
from onnxutils.quantization.quantizer import BasicQuantizer
from onnxutils.quantization.fuse_qat_conv_bn_relu import FuseQatConvBNReLU

from onnxutils.onnx2torch.scatter_nd import TorchScatterNd
from onnxutils.onnx2torch.converter import normalize_module_name

from qnn_quantizer import QnnQuantizer

In [6]:
model_quanized = symbolic_trace(torch_model, skipped_module_classes=[TorchScatterNd])

quantizer = QnnQuantizer()
model_quanized = quantizer.quantize(model_quanized)
model_quanized.print_readable()

/home/ycgao/Workdir/psurvey/tools/onnxutils/onnxutils/onnx2torch/resize.py:45: UserWarning: `nn.functional.upsample_bilinear` is deprecated. Use `nn.functional.interpolate` instead.
  x = nn.functional.upsample_bilinear(x, self.sizes)


class GraphModule(torch.nn.Module):
    def forward(self, distortion_coeff, extrinsics, fused_projection, imgs, mpp, mpp_pose_state, mpp_valid, norm_intrinsics, pose, prev_bev_feats, prev_feat_stride16, prev_pose_inv, sdmap_encode, sdmap_mat):
        # No stacktrace found for following nodes
        fq0 = self.fq0(imgs);  imgs = None
        fq1 = self.fq1(mpp);  mpp = None
        fq2 = self.fq2(sdmap_mat);  sdmap_mat = None
        
         # File: /opt/miniconda3/lib/python3.10/site-packages/torch/fx/proxy.py:219 in create_proxy, code: proxy.node.stack_trace = ''.join(CapturedTraceback.extract().format())
        mppmodule_encoder_backbone_stem_stem_0_conv = getattr(self, "MPPModule_encoder/backbone/stem/stem/0/Conv")(fq1);  fq1 = None
        
         # File: /opt/miniconda3/lib/python3.10/site-packages/torch/fx/proxy.py:219 in create_proxy, code: proxy.node.stack_trace = ''.join(CapturedTraceback.extract().format())
        random_a1a6dc5a_cc40_11ef_a198_cdeed2944927_64 = self.

'class GraphModule(torch.nn.Module):\n    def forward(self, distortion_coeff, extrinsics, fused_projection, imgs, mpp, mpp_pose_state, mpp_valid, norm_intrinsics, pose, prev_bev_feats, prev_feat_stride16, prev_pose_inv, sdmap_encode, sdmap_mat):\n        # No stacktrace found for following nodes\n        fq0 = self.fq0(imgs);  imgs = None\n        fq1 = self.fq1(mpp);  mpp = None\n        fq2 = self.fq2(sdmap_mat);  sdmap_mat = None\n        \n         # File: /opt/miniconda3/lib/python3.10/site-packages/torch/fx/proxy.py:219 in create_proxy, code: proxy.node.stack_trace = \'\'.join(CapturedTraceback.extract().format())\n        mppmodule_encoder_backbone_stem_stem_0_conv = getattr(self, "MPPModule_encoder/backbone/stem/stem/0/Conv")(fq1);  fq1 = None\n        \n         # File: /opt/miniconda3/lib/python3.10/site-packages/torch/fx/proxy.py:219 in create_proxy, code: proxy.node.stack_trace = \'\'.join(CapturedTraceback.extract().format())\n        random_a1a6dc5a_cc40_11ef_a198_cdeed29

In [7]:
# basic calibration

dataloader = DataLoader(
    DatasetUtils.take_front(
        DatasetUtils.transform(
            dataset,
            lambda item: tuple(x.cuda() for x in item)
        ),
        100),
    batch_size=None
)
model_quanized.cuda()
for data in tqdm(dataloader):
    model_quanized(*data)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [10:16<00:00,  6.17s/it]


# Analysis

In [8]:
from onnxutils.quantization.metric import compute_metrics, print_stats

def model_infer(model, data):
    model.eval()
    with torch.no_grad():
        vals = model(*data)
    if not isinstance(vals, (tuple, list)):
        vals = (vals,)
    return {name: val for name, val in zip(model.onnx_mapping.outputs, vals)}

In [9]:
model_quanized.onnx_mapping = torch_model.onnx_mapping

model_quanized.cuda()
torch_model.cuda()
dataloader = DataLoader(
    DatasetUtils.take_front(
        DatasetUtils.transform(
            dataset,
            lambda item: tuple(x.cuda() for x in item)
        ),
        10),
    batch_size=None
)

for data in tqdm(dataloader):
    real = model_infer(torch_model, data)
    pred = model_infer(model_quanized, data)
    analysis_reports = [compute_metrics(real, pred, metrics=['snr', 'mse', 'cosine'])]
    print('--------------------------------------------------------------------')
    print_stats(analysis_reports, sorted_metric='snr', reversed_order=True)

/home/ycgao/Workdir/psurvey/tools/onnxutils/onnxutils/onnx2torch/resize.py:45: UserWarning: `nn.functional.upsample_bilinear` is deprecated. Use `nn.functional.interpolate` instead.                          | 0/10 [00:00<?, ?it/s]
  x = nn.functional.upsample_bilinear(x, self.sizes)
 10%|███████████████████▍                                                                                                                                                                              | 1/10 [00:26<03:59, 26.65s/it]

--------------------------------------------------------------------
weighted_cls_scores_flat {'snr': [2.3701484203338623], 'mse': [0.007096365559846163], 'cosine': [0.06819318979978561]}
mpp_pose_fuse_state {'snr': [nan], 'mse': [nan], 'cosine': [nan]}
smtc_offset_1 {'snr': [1.7549651861190796], 'mse': [1.9472925662994385], 'cosine': [0.1831236332654953]}
roadmask_dir_out {'snr': [0.8692793250083923], 'mse': [0.12637867033481598], 'cosine': [0.5701035261154175]}
mpp_pred_t {'snr': [nan, nan], 'mse': [nan, nan], 'cosine': [nan, nan]}
smtc_offset_3 {'snr': [1.7946759462356567], 'mse': [2.073148488998413], 'cosine': [0.17699672281742096]}
gct_sigma_dt {'snr': [0.8538991808891296], 'mse': [0.9877336025238037], 'cosine': [0.6096482872962952]}
mpp_pred_r {'snr': [nan, nan], 'mse': [nan, nan], 'cosine': [nan, nan]}
roadmask_id {'snr': [1.6910072565078735], 'mse': [1.8234294652938843], 'cosine': [0.17671611905097961]}
smtc_offset_2 {'snr': [1.4855784177780151], 'mse': [1.5701695680618286], 'c

 20%|██████████████████████████████████████▊                                                                                                                                                           | 2/10 [00:53<03:33, 26.67s/it]

--------------------------------------------------------------------
refline_instance_confidence {'snr': [4.735977649688721], 'mse': [0.09993965923786163], 'cosine': [0.5770003199577332]}
weighted_cls_scores_flat {'snr': [2.25373911857605], 'mse': [0.0068908873945474625], 'cosine': [0.0532623827457428]}
bev2d_semantic_cls_result {'snr': [0.6579120755195618], 'mse': [8.899765968322754], 'cosine': [0.6777127981185913]}
mpp_pose_fuse_state {'snr': [nan], 'mse': [nan], 'cosine': [nan]}
smtc_offset_1 {'snr': [1.630687952041626], 'mse': [1.7374011278152466], 'cosine': [0.15090756118297577]}
mpp_pred_t {'snr': [nan, nan], 'mse': [nan, nan], 'cosine': [nan, nan]}
refline_instance_close_confidence {'snr': [3.3124144077301025], 'mse': [0.010737888514995575], 'cosine': [0.7183283567428589]}
mpp_pred_r {'snr': [nan, nan], 'mse': [nan, nan], 'cosine': [nan, nan]}
roadmask_id {'snr': [1.5948352813720703], 'mse': [1.7339863777160645], 'cosine': [0.1840212643146515]}
smtc_offset_3 {'snr': [1.593368530

 30%|██████████████████████████████████████████████████████████▏                                                                                                                                       | 3/10 [01:22<03:14, 27.72s/it]

...

--------------------------------------------------------------------
refline_instance_confidence {'snr': [10.862922668457031], 'mse': [0.6204457879066467], 'cosine': [0.5245333909988403]}
weighted_cls_scores_flat {'snr': [2.305739164352417], 'mse': [0.00707270298153162], 'cosine': [0.059154219925403595]}
mpp_pose_fuse_state {'snr': [nan], 'mse': [nan], 'cosine': [nan]}
smtc_offset_1 {'snr': [1.6665292978286743], 'mse': [2.1189351081848145], 'cosine': [0.16548475623130798]}
roadmask_dir_out {'snr': [0.8544617295265198], 'mse': [0.12720149755477905], 'cosine': [0.5624265670776367]}
mpp_pred_t {'snr': [nan, nan], 'mse': [nan, nan], 'cosine': [nan, nan]}
smtc_offset_2 {'snr': [1.9368469715118408], 'mse': [2.3935132026672363], 'cosine': [0.10739819705486298]}
smtc_offset_3 {'snr': [1.4181733131408691], 'mse': [1.7861204147338867], 'cosine': [0.16475242376327515]}
mpp_pred_r {'snr': [nan, nan], 'mse': [nan, nan], 'cosine': [nan, nan]}
roadmask_id {'snr': [1.771567463874817], 'mse': [2.116446

 90%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                   | 9/10 [04:19<00:29, 29.18s/it]

--------------------------------------------------------------------
weighted_cls_scores_flat {'snr': [2.2055487632751465], 'mse': [0.007020102348178625], 'cosine': [0.06307191401720047]}
mpp_pose_fuse_state {'snr': [nan], 'mse': [nan], 'cosine': [nan]}
mpp_pred_t {'snr': [nan, nan], 'mse': [nan, nan], 'cosine': [nan, nan]}
smtc_offset_3 {'snr': [1.731014609336853], 'mse': [1.8025541305541992], 'cosine': [0.12041768431663513]}
smtc_offset_2 {'snr': [1.6222065687179565], 'mse': [1.9536547660827637], 'cosine': [0.1444815695285797]}
smtc_offset_1 {'snr': [1.5800354480743408], 'mse': [1.7822405099868774], 'cosine': [0.19150583446025848]}
det3d_decoded_bboxes {'snr': [0.8111751079559326], 'mse': [630.14892578125], 'cosine': [0.6065347194671631]}
roadmask_dir_out {'snr': [0.8069505095481873], 'mse': [0.11914204806089401], 'cosine': [0.5849778652191162]}
gct_sigma_dt {'snr': [0.773856520652771], 'mse': [0.8124505877494812], 'cosine': [0.6040159463882446]}
marker_class_softmax {'snr': [0.67133

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [04:50<00:00, 29.06s/it]

--------------------------------------------------------------------
refline_instance_confidence {'snr': [12.957855224609375], 'mse': [0.7763251066207886], 'cosine': [0.427953839302063]}
weighted_cls_scores_flat {'snr': [2.162576913833618], 'mse': [0.0062704929150640965], 'cosine': [0.05471578240394592]}
mpp_pose_fuse_state {'snr': [nan], 'mse': [nan], 'cosine': [nan]}
smtc_offset_1 {'snr': [1.7316700220108032], 'mse': [2.0883891582489014], 'cosine': [0.20348641276359558]}
roadmask_dir_out {'snr': [0.8335930109024048], 'mse': [0.12501339614391327], 'cosine': [0.5746960639953613]}
mpp_pred_t {'snr': [nan, nan], 'mse': [nan, nan], 'cosine': [nan, nan]}
smtc_offset_2 {'snr': [2.2221736907958984], 'mse': [2.5600874423980713], 'cosine': [0.07936227321624756]}
smtc_offset_3 {'snr': [1.3182716369628906], 'mse': [1.7986984252929688], 'cosine': [0.16689273715019226]}
mpp_pred_r {'snr': [nan, nan], 'mse': [nan, nan], 'cosine': [nan, nan]}
roadmask_id {'snr': [1.7702726125717163], 'mse': [2.04774

# Export

In [10]:
model_finalized = quantizer.finalize(model_quanized)
torch.onnx.export(
    model_finalized,
    tuple(next(iter(dataloader))),
    qmodel_path,
    input_names=torch_model.onnx_mapping.inputs,
    output_names=torch_model.onnx_mapping.outputs,
)

/home/ycgao/Workdir/psurvey/tools/onnxutils/onnxutils/onnx2torch/scatter_nd.py:38: TracerWarning: Converting a tensor to a Python list might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  output_indices = indices.reshape((-1, indices.shape[-1])).T.tolist()
/opt/miniconda3/lib/python3.10/site-packages/torch/onnx/symbolic_opset9.py:1959: FutureWarning: 'torch.onnx.symbolic_opset9._cast_Bool' is deprecated in version 2.0 and will be removed in the future. Please Avoid using this function and create a Cast node instead.
  return fn(g, to_cast_func(g, input, False), to_cast_func(g, other, False))
/opt/miniconda3/lib/python3.10/site-packages/torch/onnx/_internal/jit_utils.py:308: UserWarning: Constant folding - Only steps=1 can be constant folded for opset >= 10 onnx::Slice op. Constant folding not applied. (Triggered internally at ../